### <center>**Prompt Template**</center>

In [1]:
GROQ_API_KEY="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"
import os
from dotenv import load_dotenv
load_dotenv()

groq_api_key=os.getenv("GROQ_API_KEY")

## Calling our model
from langchain_openai import ChatOpenAI
from langchain_groq import ChatGroq

model = ChatGroq(model="llama-3.3-70b-specdec", groq_api_key=GROQ_API_KEY)
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x1267307d0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x125e59c90>, model_name='llama-3.3-70b-specdec', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [5]:
## 
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder
prompt = ChatPromptTemplate.from_messages(
    [
        ("system","You are a Helpful assistant. Answer all the questions of the nest of your ability"),
        MessagesPlaceholder(variable_name="messages")
    ]
)
chain = prompt|model

In [13]:
from langchain_core.messages import HumanMessage
chain.invoke({"messages":[HumanMessage(content="Hi I am Surya Vishal")]})


AIMessage(content="Hello Surya Vishal! It's nice to meet you. Is there something I can help you with or would you like to chat? I'm here to assist you with any questions or topics you'd like to discuss. How's your day going so far?", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 55, 'prompt_tokens': 59, 'total_tokens': 114, 'completion_time': 0.024657121, 'prompt_time': 0.006010965, 'queue_time': 0.235200534, 'total_time': 0.030668086}, 'model_name': 'llama-3.3-70b-specdec', 'system_fingerprint': 'fp_9f325da874', 'finish_reason': 'stop', 'logprobs': None}, id='run-3e4fd8f6-9337-4791-9aba-d3d0d75bb29b-0', usage_metadata={'input_tokens': 59, 'output_tokens': 55, 'total_tokens': 114})

In [15]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}

def get_session_history(session_id:str)-> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

with_message_history=RunnableWithMessageHistory(chain,get_session_history)

In [16]:
config = {"configurable":{"session_id":"chat3"}}
response=with_message_history.invoke(
    [HumanMessage(content="Hi I am Surya Vishal")],
    config=config
)

response

AIMessage(content="Hello Surya Vishal! It's nice to meet you. Is there something I can help you with or would you like to chat? I'm here to assist you with any questions or topics you'd like to discuss. How's your day going so far?", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 55, 'prompt_tokens': 59, 'total_tokens': 114, 'completion_time': 0.024550065, 'prompt_time': 0.005980725, 'queue_time': 0.235418242, 'total_time': 0.03053079}, 'model_name': 'llama-3.3-70b-specdec', 'system_fingerprint': 'fp_9f325da874', 'finish_reason': 'stop', 'logprobs': None}, id='run-aace8f52-2ec6-42bd-8792-55f9425ad646-0', usage_metadata={'input_tokens': 59, 'output_tokens': 55, 'total_tokens': 114})

In [17]:
## Add more complexity 
prompt = ChatPromptTemplate.from_messages(
    [
        ("system","You are a Helpful assistant. Answer all the questions of the nest of your ability in {languages}"),
        MessagesPlaceholder(variable_name="messages"),
    ]
)
chain = prompt|model

In [18]:
name = input("Enter name: ")
response = chain.invoke({"messages":[HumanMessage(content=f"Hi my name is {name}")],"languages":"Telugu"})
response.content

'హాయ్ సూర్య విషాల్, నమస్తే! నేను మీకు ఎలా సహాయం చేయగలను?'

Let's now wrap this more complicated chain in a Message History class. This time, because there are multiple keys in the input, we need to specify the correct key to use to save the chat history.

In [19]:
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages"
)

In [20]:
config={"configurable":{"session_id":"chat4"}}
response=with_message_history.invoke(
    {'messages':[HumanMessage(content=f"Hii I am {name}")],"languages":"Telugu"},
    config=config
)
response.content

'నమస్కారం సూర్య విశాల్ గారూ! మీరు ఎలా ఉన్నారు? నా ద్వారా మీకు ఏదైనా సహాయం అవసరమా? మా సంభాషణ కొనసాగిద్దాం!'

In [21]:
config={"configurable":{"session_id":"chat4"}}
response=with_message_history.invoke(
    {'messages':[HumanMessage(content=f"ஹாய், நான் சூர்யா விஷால்.")],"languages":"Telugu"},
    config=config
)
response.content

'అరే, నేను తెలుగులో మాట్లాడుతున్నాను, కానీ మీరు తమిళంలో మాట్లాడుతున్నారు. తమిళం మరియు తెలుగు రెండూ భారతదేశంలో మాట్లాడే భాషలు, కానీ అవి ఒకదానికొకటి భిన్నంగా ఉంటాయి.\n\nమీరు తెలుగులో మాట్లాడటం ప్రారంభించాలనుకుంటే, నేను సంతోషంగా మీతో మాట్లాడతాను. లేకపోతే, నేను తమిళంలో మాట్లాడటానికి ప్రయత్నించగలను, కానీ నా తమిళ భాషా నైపుణ్యం పరిమితంగా ఉంటుంది.\n\nమీరు ఏమి చెప్పాలనుకుంటున్నారు?'